In [1]:
import requests
import json
from urllib.parse import urlencode
import pandas as pd
import math
import numpy as np

# url = 'http://data.ex.co.kr/openapi/specialAnal/intercityLeadTime?key=6723386324&type=xml&iYear=2018&numOfRows=10&pageNo=1'
# response = requests.get(url)

# print(response)

In [76]:
test1 = pd.read_csv('./data/교통량/DSRC_특송기간+총통행시간_1년_1년_2013', encoding='cp949')
test1.head()

,구분,특수일전후특송기간범위구분코드,TCS차종구분명,TCS차종구분코드,TCS차종유형구분명,TCS차종유형구분코드,교통량,총통행시간,평균통행시간
0,D(20130101),4,1종,1,소형차,1,3565157,165845367,47
1,D(20130101),4,6종,6,소형차,1,186613,7789037,42
2,D(20130101),4,2종,2,중형차,2,69966,2228828,32
3,D(20130101),4,3종,3,대형차,3,143598,7315697,51
4,D(20130101),4,5종,5,대형차,3,37464,2750942,73


In [77]:
test2 = pd.read_csv('./data/교통량/DSRC_특송기간+총통행시간_1년_1년_2019', encoding='cp949')
test2.head()

,구분,특수일전후특송기간범위구분코드,TCS차종구분명,TCS차종구분코드,TCS차종유형구분명,TCS차종유형구분코드,교통량,총통행시간,평균통행시간
0,D(20190101),4,1종,1,소형차,1,5612988,184287796,33
1,D(20190101),4,6종,6,소형차,1,268856,8258891,31
2,D(20190101),4,2종,2,중형차,2,70769,1489281,21
3,D(20190101),4,4종,4,대형차,3,56796,1903066,34
4,D(20190101),4,3종,3,대형차,3,158520,5452960,34


In [34]:
test = pd.read_csv('./data/교통량/DSRC_특송기간+총통행시간_1년_1년_2012', encoding='cp949')
test

,구분,특수일전후특송기간범위구분코드,TCS차종구분명,TCS차종구분코드,TCS차종유형구분명,TCS차종유형구분코드,교통량,총통행시간,평균통행시간
0,D(20120101),4,1종,1,소형차,1,2157331,0,0
1,D(20120101),4,6종,6,소형차,1,106342,0,0
2,D(20120101),4,2종,2,중형차,2,20938,0,0
3,D(20120101),4,4종,4,대형차,3,11310,0,0
4,D(20120101),4,3종,3,대형차,3,63993,0,0
...,...,...,...,...,...,...,...,...,...
121,D-3(20120927),1,1종,1,소형차,1,4396395,140898714,32
122,D-3(20120927),1,2종,2,중형차,2,308605,11555354,37
123,D-3(20120927),1,3종,3,대형차,3,292254,13556276,46
124,D-3(20120927),1,5종,5,대형차,3,308414,23141081,75


2013년부터 평균 통행시간에 대한 데이터가 정상적으로 있으므로 2013년 데이터 부터 사용하도록 한다.

In [35]:
import glob
traffic_list = glob.glob('./data/교통량/*')

In [41]:
sorted(traffic_list)[3:]

['./data/교통량/DSRC_특송기간+총통행시간_1년_1년_2013',
 './data/교통량/DSRC_특송기간+총통행시간_1년_1년_2014',
 './data/교통량/DSRC_특송기간+총통행시간_1년_1년_2015',
 './data/교통량/DSRC_특송기간+총통행시간_1년_1년_2016',
 './data/교통량/DSRC_특송기간+총통행시간_1년_1년_2017',
 './data/교통량/DSRC_특송기간+총통행시간_1년_1년_2018',
 './data/교통량/DSRC_특송기간+총통행시간_1년_1년_2019',
 './data/교통량/DSRC_특송기간+총통행시간_1년_1년_2020',
 './data/교통량/DSRC_특송기간+총통행시간_1년_1년_2021']

In [46]:
with open('./data/교통량/traffic_merged.csv', 'w', encoding='cp949') as outfile:
  for filename in sorted(traffic_list)[3:]:
    with open(filename, encoding='cp949') as file:
      for line in file:
        outfile.write(line)

In [47]:
traffic_merged = pd.read_csv('./data/교통량/traffic_merged.csv', encoding='cp949')

In [65]:
traffic_merged['구분'][0][-9:-1]

'20130101'

In [67]:
traffic_merged['구분'].loc[0]

'D(20130101)'

In [68]:
for i in range(traffic_merged.shape[0]):
  traffic_merged['구분'].loc[i] = traffic_merged['구분'][i][-9:-1]

In [69]:
traffic_merged['구분'] = traffic_merged['구분'] + traffic_merged['특수일전후특송기간범위구분코드']

0       20130101
1       20130101
2       20130101
3       20130101
4       20130101
          ...   
1107    20210918
1108    20210918
1109    20210918
1110    20210918
1111    20210918
Name: 구분, Length: 1112, dtype: object

하지만 이렇게 다운로드된 파일은 단위 시간에 대한 정보가 없고 갖고 있는 정보량이 한정적이다..

In [105]:
# temp_df = pd.DataFrame(columns=["출발지", "기준시간","방향" ,str(Year)+"소요시간"])

url = 'http://data.ex.co.kr/openapi/specialAnal/intercityLeadTime'
parameters = {"key":"6723386324", "type":"xml", "numOfRows":"24", "pageNo":'216', "iYear":'2018'}
req = requests.get(url, params=parameters)

xml_data = xmltodict.parse(req.text)
informations = xml_data['data']['intercityLeadTimeLists']

if informations[0]['dirNm'] is not None :
  print('True')
else:
  print('false')

True


In [131]:
import xmltodict

def getReturnTimeData(startPageNo, endPageNo, Year):
  info_df_daegeon = pd.DataFrame(columns=["when","date","year", "hour", "destination","방향" ,"spend_time"])
  info_df_daegu = pd.DataFrame(columns=["when","date","year", "hour", "destination","방향", "spend_time"])
  info_df_busan = pd.DataFrame(columns=["when","date","year", "hour", "destination","방향","spend_time"])
  info_df_gwangju = pd.DataFrame(columns=["when","date","year", "hour", "destination","방향","spend_time"])
  info_df_ganglung = pd.DataFrame(columns=["when","date","year", "hour", "destination","방향","spend_time"])
  info_df_mockpo = pd.DataFrame(columns=["when","date","year", "hour", "destination","방향","spend_time"])
  
  daegeon = 0
  daegu = 0
  busan = 0
  gwangju = 0
  ganglung = 0
  mockpo = 0

  for pageNo in range(startPageNo, endPageNo):
    url = 'http://data.ex.co.kr/openapi/specialAnal/intercityLeadTime'
    parameters = {"key":"6723386324", "type":"xml", "numOfRows":"24", "pageNo":str(pageNo), "iYear":str(Year)}
    req = requests.get(url, params=parameters)

    xml_data = xmltodict.parse(req.text)
    informations = xml_data['data']['intercityLeadTimeLists']

    for info in informations:
      if (info['dirNm'] == '귀성방향') and (info['stUnitNm'] == '서울'):
        try:
          if info['edUnitNm'] == "대전":
            info_df_daegeon.loc[daegeon] = [info['sphlDfttNm'], info['sphlDfttScopTypeNm'],info['stdYear'], info['stdHour'], info['edUnitNm'], info['dirNm'], float(info['trtm'][0:5])]
            daegeon += 1
          elif info['edUnitNm'] == "대구":
            info_df_daegu.loc[daegu] = [info['sphlDfttNm'], info['sphlDfttScopTypeNm'],info['stdYear'], info['stdHour'], info['edUnitNm'], info['dirNm'], float(info['trtm'][0:5])]
            daegu += 1
          elif info['edUnitNm'] == "부산":
            info_df_busan.loc[busan] = [info['sphlDfttNm'], info['sphlDfttScopTypeNm'],info['stdYear'], info['stdHour'], info['edUnitNm'], info['dirNm'], float(info['trtm'][0:5])]
            busan += 1
          elif info['edUnitNm'] == "광주":
            info_df_gwangju.loc[gwangju] = [info['sphlDfttNm'], info['sphlDfttScopTypeNm'],info['stdYear'], info['stdHour'], info['edUnitNm'], info['dirNm'], float(info['trtm'][0:5])]
            gwangju += 1
          elif info['edUnitNm'] == "강릉":
            info_df_ganglung.loc[ganglung] = [info['sphlDfttNm'], info['sphlDfttScopTypeNm'],info['stdYear'], info['stdHour'], info['edUnitNm'], info['dirNm'], float(info['trtm'][0:5])]
            ganglung += 1
          elif info['edUnitNm'] == "목포":
            info_df_mockpo.loc[mockpo] = [info['sphlDfttNm'], info['sphlDfttScopTypeNm'],info['stdYear'], info['stdHour'], info['edUnitNm'], info['dirNm'], float(info['trtm'][0:5])]
            mockpo += 1
        except:
          continue
  return (info_df_daegeon, info_df_daegu, info_df_busan, info_df_gwangju, info_df_ganglung, info_df_mockpo)


In [137]:
data_2017_full = getReturnTimeData(1, 85, 2017)
data_2018_full = getReturnTimeData(132, 217, 2018)
data_2019_full = getReturnTimeData(131, 191, 2019)
data_2020_full = getReturnTimeData(130, 213, 2020)
data_2021_full = getReturnTimeData(129, 211, 2021)

In [ ]:
# data_2017_full.to_csv('./data/데이터셋/2017_full.to_csv', sep=',', na_rep='NaN')
# # data_2018_full.to_csv('./data/데이터셋/2018_full.to_csv', sep=',', na_rep='NaN')
# # data_2019_full.to_csv('./data/데이터셋/2019_full.to_csv', sep=',', na_rep='NaN')
# # data_2020_full.to_csv('./data/데이터셋/2020_dday.to_csv', sep=',', na_rep='NaN')
# # data_2021_full.to_csv('./data/데이터셋/2021_full.to_csv', sep=',', na_rep='NaN')

In [140]:
data_2017_deageon = data_2017_full[0]
data_2017_daegu = data_2017_full[1]
data_2017_busan = data_2017_full[2]
data_2017_gwangju = data_2017_full[3]
data_2017_ganglung = data_2017_full[4]
data_2017_mockpo = data_2017_full[5]

In [141]:
data_2017_deageon.to_csv('./data/데이터셋/data_2017_deageon.csv', sep=',', na_rep='NaN')
data_2017_daegu.to_csv('./data/데이터셋/data_2017_daegu.csv', sep=',', na_rep='NaN')
data_2017_busan.to_csv('./data/데이터셋/data_2017_busan.csv', sep=',', na_rep='NaN')
data_2017_gwangju.to_csv('./data/데이터셋/data_2017_gwangju.csv', sep=',', na_rep='NaN')
data_2017_ganglung.to_csv('./data/데이터셋/data_2017_ganglung.csv', sep=',', na_rep='NaN')
data_2017_mockpo.to_csv('./data/데이터셋/data_2017_mockpo.csv', sep=',', na_rep='NaN')

In [142]:
data_2018_deageon = data_2018_full[0]
data_2018_daegu = data_2018_full[1]
data_2018_busan = data_2018_full[2]
data_2018_gwangju = data_2018_full[3]
data_2018_ganglung = data_2018_full[4]
data_2018_mockpo = data_2018_full[5]

In [143]:
data_2018_deageon.to_csv('./data/데이터셋/data_2018_deageon.csv', sep=',', na_rep='NaN')
data_2018_daegu.to_csv('./data/데이터셋/data_2018_daegu.csv', sep=',', na_rep='NaN')
data_2018_busan.to_csv('./data/데이터셋/data_2018_busan.csv', sep=',', na_rep='NaN')
data_2018_gwangju.to_csv('./data/데이터셋/data_2018_gwangju.csv', sep=',', na_rep='NaN')
data_2018_ganglung.to_csv('./data/데이터셋/data_2018_ganglung.csv', sep=',', na_rep='NaN')
data_2018_mockpo.to_csv('./data/데이터셋/data_2018_mockpo.csv', sep=',', na_rep='NaN')

In [144]:
data_2019_deageon = data_2019_full[0]
data_2019_daegu = data_2019_full[1]
data_2019_busan = data_2019_full[2]
data_2019_gwangju = data_2019_full[3]
data_2019_ganglung = data_2019_full[4]
data_2019_mockpo = data_2019_full[5]

In [145]:
data_2019_deageon.to_csv('./data/데이터셋/data_2019_deageon.csv', sep=',', na_rep='NaN')
data_2019_daegu.to_csv('./data/데이터셋/data_2019_daegu.csv', sep=',', na_rep='NaN')
data_2019_busan.to_csv('./data/데이터셋/data_2019_busan.csv', sep=',', na_rep='NaN')
data_2019_gwangju.to_csv('./data/데이터셋/data_2019_gwangju.csv', sep=',', na_rep='NaN')
data_2019_ganglung.to_csv('./data/데이터셋/data_2019_ganglung.csv', sep=',', na_rep='NaN')
data_2019_mockpo.to_csv('./data/데이터셋/data_2019_mockpo.csv', sep=',', na_rep='NaN')

In [146]:
data_2020_deageon = data_2020_full[0]
data_2020_daegu = data_2020_full[1]
data_2020_busan = data_2020_full[2]
data_2020_gwangju = data_2020_full[3]
data_2020_ganglung = data_2020_full[4]
data_2020_mockpo = data_2020_full[5]

In [147]:
data_2020_deageon.to_csv('./data/데이터셋/data_2020_deageon.csv', sep=',', na_rep='NaN')
data_2020_daegu.to_csv('./data/데이터셋/data_2020_daegu.csv', sep=',', na_rep='NaN')
data_2020_busan.to_csv('./data/데이터셋/data_2020_busan.csv', sep=',', na_rep='NaN')
data_2020_gwangju.to_csv('./data/데이터셋/data_2020_gwangju.csv', sep=',', na_rep='NaN')
data_2020_ganglung.to_csv('./data/데이터셋/data_2020_ganglung.csv', sep=',', na_rep='NaN')
data_2020_mockpo.to_csv('./data/데이터셋/data_2020_mockpo.csv', sep=',', na_rep='NaN')

In [148]:
data_2021_deageon = data_2021_full[0]
data_2021_daegu = data_2021_full[1]
data_2021_busan = data_2021_full[2]
data_2021_gwangju = data_2021_full[3]
data_2021_ganglung = data_2021_full[4]
data_2021_mockpo = data_2021_full[5]

In [149]:
data_2021_deageon.to_csv('./data/데이터셋/data_2021_deageon.csv', sep=',', na_rep='NaN')
data_2021_daegu.to_csv('./data/데이터셋/data_2021_daegu.csv', sep=',', na_rep='NaN')
data_2021_busan.to_csv('./data/데이터셋/data_2021_busan.csv', sep=',', na_rep='NaN')
data_2021_gwangju.to_csv('./data/데이터셋/data_2021_gwangju.csv', sep=',', na_rep='NaN')
data_2021_ganglung.to_csv('./data/데이터셋/data_2021_ganglung.csv', sep=',', na_rep='NaN')
data_2021_mockpo.to_csv('./data/데이터셋/data_2021_mockpo.csv', sep=',', na_rep='NaN')

In [151]:
data_2021_busan

,when,date,year,hour,destination,방향,spend_time
0,추석,D-3,2021,00,부산,귀성방향,257.0
1,추석,D-3,2021,01,부산,귀성방향,260.9
2,추석,D-3,2021,02,부산,귀성방향,259.4
3,추석,D-3,2021,03,부산,귀성방향,253.7
4,추석,D-3,2021,04,부산,귀성방향,254.7
...,...,...,...,...,...,...,...
156,추석,D+3,2021,18,부산,귀성방향,277.7
157,추석,D+3,2021,19,부산,귀성방향,286.8
158,추석,D+3,2021,20,부산,귀성방향,301.1
159,추석,D+3,2021,21,부산,귀성방향,315.4
